**1. Reading In Books We Like**

In [1]:
import pandas as pd 

my_books = pd.read_csv("liked_books.csv", index_col=0)
my_books["book_id"] = my_books["book_id"].astype(str)

In [2]:
my_books

,user_id,book_id,rating,title
0,-1,17788401,3,Ugly Love
1,-1,60310757,5,"Lightlark (Lightlark, #1)"
2,-1,58410355,0,Nightbane (Lightlark #2)
3,-1,64645812,0,If Only I Had Told Her
4,-1,44676678,4,If He Had Been with Me
...,...,...,...,...
101,-1,23003206,0,Everything I Never Told You
102,-1,3437,1,The Curious Incident of the Dog in the Night-Time
103,-1,4378870,3,The Boy in the Striped Pajamas
104,-1,18690890,3,Paper Towns


In [3]:
my_books["book_id"] = my_books["book_id"].astype(str)

**2. Finding Similar Users**

In [4]:
csv_book_mapping = {}

with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

**Set that contains all of the unique books that we have read.**

In [5]:
book_set = set(my_books["book_id"])

**Every user that read the same book as us will be plaved in this overlap users dictionary.**

In [6]:
!wc -l goodreads_interactions.csv

 228648343 goodreads_interactions.csv


**Note: _ means that we do not care about that variable.**

In [7]:
overlap_users = {}

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        book_id = csv_book_mapping.get(csv_id)
        
        if book_id in book_set:
            if user_id not in overlap_users:
                overlap_users[user_id] = 1
            else:
                overlap_users[user_id] += 1

In [8]:
filtered_overlap_users = set([k for k in overlap_users if overlap_users[k] > my_books.shape[0]/5])

**3. Finding Similar User Book Ratings**

In [9]:
interactions_list = []

with open("goodreads_interactions.csv") as f:
    while True:
        line = f.readline()
        if not line:
            break

        user_id, csv_id, _, rating, _ = line.strip().split(",")
        if user_id in filtered_overlap_users:
            book_id = csv_book_mapping[csv_id]
            interactions_list.append([user_id, book_id, rating]) 

**4. Creating A User /Book Matrix**

In [10]:
len(interactions_list)

1620249

In [11]:
interactions_list[0]

['281', '6216670', '5']

**DateFrame**

In [12]:
interactions = pd.DataFrame(interactions_list, columns=["user_id", "book_id", "rating"])

In [13]:
interactions = pd.concat([my_books[["user_id", "book_id", "rating"]], interactions])

In [14]:
interactions

,user_id,book_id,rating
0,-1,17788401,3
1,-1,60310757,5
2,-1,58410355,0
3,-1,64645812,0
4,-1,44676678,4
...,...,...,...
1620244,441283,35068798,0
1620245,441283,34227692,0
1620246,441283,32969999,0
1620247,441283,23705532,0


In [15]:
interactions["book_id"] = interactions["book_id"].astype(str)
interactions["user_id"] = interactions["user_id"].astype(str)
interactions["rating"] = pd.to_numeric(interactions["rating"])

In [16]:
interactions["user_id"].unique()

array(['-1', '281', '1033', '2794', '3012', '7488', '9959', '13856',
       '22124', '22864', '23326', '28128', '34134', '40239', '45510',
       '51611', '55191', '65620', '66585', '77736', '79624', '83250',
       '83293', '83470', '89997', '91247', '93625', '96010', '99975',
       '100939', '105106', '106001', '113970', '116512', '119660',
       '121119', '122431', '126987', '130849', '132126', '137133',
       '138588', '141311', '143201', '145486', '146601', '147132',
       '154116', '155631', '157964', '165733', '171930', '173042',
       '173804', '177877', '180813', '180817', '182379', '183847',
       '188693', '191515', '192434', '192632', '193311', '195488',
       '198123', '198161', '199610', '199839', '200059', '205784',
       '206213', '206369', '207960', '207969', '209647', '210169',
       '210394', '212288', '213935', '214546', '214711', '214914',
       '215878', '216545', '217460', '219243', '219898', '219965',
       '220824', '220848', '222577', '223539', '223

**Note: cat.codes means category codes.**

In [17]:
interactions["user_index"] = interactions["user_id"].astype("category").cat.codes

**User ID and Book Id - To Position.**

In [18]:
interactions["user_index"].unique()

array([  0, 103,   2,  99, 116, 187, 197,  14,  65,  70,  71, 105, 138,
       166, 182, 183, 184, 185, 186, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 198,   1,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        66,  67,  68,  69,  72,  73,  74,  75,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98, 100, 101, 102, 104, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150,
       151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 16

In [19]:
interactions["book_index"] = interactions["book_id"].astype("category").cat.codes

**Note 1: Sparse Matrix will be used.**

**Note 2: if there is no value in a column you just leave it blank and not take up any memory or storage space (Sparse Matrix).**

In [20]:
from scipy.sparse import coo_matrix

ratings_mat_coo = coo_matrix((interactions["rating"], (interactions["user_index"], interactions["book_index"])))

In [21]:
ratings_mat_coo

<199x461150 sparse matrix of type '<class 'numpy.int64'>'
	with 1620354 stored elements in COOrdinate format>

**Convert coo matrix to csr matrix.**

**coo matrices are a little bit easier to create which is why we initially created it in coo format and now we are going to convert it in csr format.**

In [22]:
ratings_mat = ratings_mat_coo.tocsr()

**5. Finding Users Similar To Us**

In [23]:
interactions[interactions["user_id"] == "-1"]

,user_id,book_id,rating,user_index,book_index
0,-1,17788401,3,0,116370
1,-1,60310757,5,0,387557
2,-1,58410355,0,0,383990
3,-1,64645812,0,0,398594
4,-1,44676678,4,0,366641
...,...,...,...,...,...
101,-1,23003206,0,0,200857
102,-1,3437,1,0,341345
103,-1,4378870,3,0,365383
104,-1,18690890,3,0,142559


In [24]:
my_index = 0

**Use a cosine similarity measure to find users that are similar to us and have similar taste in books.**

**Cosine similarity will just find the similarity between two rows in our matrix so that we can find how similar each user is to us in terms of what books they read and how they rated them.**

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(ratings_mat[my_index,:], ratings_mat).flatten()

In [26]:
similarity[0]

1.0000000000000007

**We will find the indices of the users who are most similar to us - That is what this numpy r partition function does - We're passing in negative 15. So what we're going to find are the 15 users who have the most similar taste.**

In [27]:
import numpy as np

indices = np.argpartition(similarity, -15)[-15:]

In [28]:
indices

array([ 90,  51,  98,  45,  72,  42,  89,  64, 132, 129,  83,  67, 104,
        93,   0])

**Find all of the rows in interactions where the user index is in our indices.**

In [29]:
similar_users = interactions[interactions["user_index"].isin(indices)].copy()

**We'll just take ourselves out so we do not get book recommendations from ourselves.**

In [30]:
similar_users = similar_users[similar_users["user_id"]!="-1"]

In [31]:
similar_users

,user_id,book_id,rating,user_index,book_index
675471,199839,270807,4,42,271119
675472,199839,28194,4,42,282944
675473,199839,16068905,4,42,87673
675474,199839,6186357,4,42,391168
675475,199839,7631105,4,42,421656
...,...,...,...,...,...
1093408,328392,28259094,0,132,284212
1093409,328392,22433390,0,132,188235
1093410,328392,29386918,0,132,296293
1093411,328392,32320661,0,132,325012


**6. Creating Book Recommendations**

**How many times each book appeared in these recommendations.**

In [32]:
book_recs = similar_users.groupby("book_id").rating.agg(['count', 'mean'])

In [33]:
book_recs

,count,mean
book_id,,
1,7,2.714286
10000191,1,0.000000
1000239,1,0.000000
1000392,1,0.000000
10016013,1,0.000000
...,...,...
9975679,2,0.000000
9978196,1,0.000000
998,1,0.000000


**Adding book titles.**

In [34]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

**Merge our 2 datasets to get the book titles into our recommendations.**

In [35]:
book_recs = book_recs.merge(books_titles, how="inner", on="book_id")

In [36]:
book_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title
0,1,7,2.714286,Harry Potter and the Half-Blood Prince (Harry ...,1713866,https://www.goodreads.com/book/show/1.Harry_Po...,https://images.gr-assets.com/books/1361039191m...,harry potter and the halfblood prince harry po...
1,10000191,1,0.000000,Yellow Crocus,17787,https://www.goodreads.com/book/show/10000191-y...,https://s.gr-assets.com/assets/nophoto/book/11...,yellow crocus
2,1000239,1,0.000000,My Secret War: The World War II Diary of Madel...,3189,https://www.goodreads.com/book/show/1000239.My...,https://s.gr-assets.com/assets/nophoto/book/11...,my secret war the world war ii diary of madeli...
3,1000392,1,0.000000,"Naruto, Vol. 16: Eulogy (Naruto, #16)",5837,https://www.goodreads.com/book/show/1000392.Na...,https://images.gr-assets.com/books/1435525315m...,naruto vol 16 eulogy naruto 16
4,10016013,1,0.000000,Harry Potter and the Methods of Rationality,8691,https://www.goodreads.com/book/show/10016013-h...,https://images.gr-assets.com/books/1293582551m...,harry potter and the methods of rationality
...,...,...,...,...,...,...,...,...
11964,9975679,2,0.000000,"Ashes (Ashes Trilogy, #1)",19788,https://www.goodreads.com/book/show/9975679-ashes,https://s.gr-assets.com/assets/nophoto/book/11...,ashes ashes trilogy 1
11965,9978196,1,0.000000,A Midsummer Tights Dream (The Misadventures of...,1785,https://www.goodreads.com/book/show/9978196-a-...,https://images.gr-assets.com/books/1327429845m...,a midsummer tights dream the misadventures of ...
11966,998,1,0.000000,The Millionaire Next Door: The Surprising Secr...,46032,https://www.goodreads.com/book/show/998.The_Mi...,https://s.gr-assets.com/assets/nophoto/book/11...,the millionaire next door the surprising secre...
11967,998578,1,4.000000,"Batgirl, Vol. 5: Kicking Assassins",249,https://www.goodreads.com/book/show/998578.Bat...,https://s.gr-assets.com/assets/nophoto/book/11...,batgirl vol 5 kicking assassins


**7. Ranking Our Book Recommendations**

**Figuring out which recommendations are relevant to us.**

**We'll first create an adjusted count which is a count but normalized for how many times the book appeared among people similar to us.**

**Ratings is the number of times the book was rated across all of goodreads.**

In [37]:
book_recs["adjusted_count"] = book_recs["count"] * (book_recs["count"] / book_recs["ratings"])

**Score: Indicating how much we might like a book.**

In [38]:
book_recs["score"] = book_recs["mean"] * book_recs["adjusted_count"]

**Take out any books we already have read.**

**1st we'll take out any book where the book id matches an id of a book we already read. The 2nd thing we'll do is we'll take out any books where the title matches a book we've already read.**

In [39]:
book_recs = book_recs[~book_recs["book_id"].isin(my_books["book_id"])]

In [40]:
my_books["mod_title"] = my_books["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True).str.lower()

**Replace any sequences of spaces.**

In [41]:
my_books["mod_title"] = my_books["mod_title"].str.replace("\s+", " ", regex=True)

**Let's take out anything in our recommendations where the mod title fits into the books we already read.**

In [42]:
book_recs = book_recs[~book_recs["mod_title"].isin(my_books["mod_title"])]

**Remove anything that appeared less than twice or less in our recommendations.**

In [43]:
book_recs = book_recs[book_recs["count"]>2]

**We want to make sure that we only find books where our mean rating is greater than a certain amount.**

In [44]:
book_recs = book_recs[book_recs["mean"] >=4]

**Creating our top recommendations which we'll sort based on our score (This basically sorts the dataframe based on the score).**

In [45]:
top_recs = book_recs.sort_values("mean", ascending=False)

In [46]:
top_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score
1339,13455782,4,4.500000,"Shatter Me (Shatter Me, #1)",14847,https://www.goodreads.com/book/show/13455782-s...,https://images.gr-assets.com/books/1338924054m...,shatter me shatter me 1,0.001078,0.004849
3174,1809465,3,4.333333,"Rodrick Rules (Diary of a Wimpy Kid, #2)",95364,https://www.goodreads.com/book/show/1809465.Ro...,https://images.gr-assets.com/books/1360095964m...,rodrick rules diary of a wimpy kid 2,0.000094,0.000409
8748,3293821,3,4.333333,"The Last Straw (Diary of a Wimpy Kid, #3)",85757,https://www.goodreads.com/book/show/3293821-th...,https://images.gr-assets.com/books/1350396036m...,the last straw diary of a wimpy kid 3,0.000105,0.000455
7230,2767052,13,4.307692,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...,https://images.gr-assets.com/books/1447303603m...,the hunger games the hunger games 1,0.000034,0.000149
3031,17927395,7,4.285714,A Court of Mist and Fury (A Court of Thorns an...,120403,https://www.goodreads.com/book/show/17927395-a...,https://images.gr-assets.com/books/1485259138m...,a court of mist and fury a court of thorns and...,0.000407,0.001744
1268,13335037,9,4.111111,"Divergent (Divergent, #1)",1962813,https://www.goodreads.com/book/show/13335037-d...,https://images.gr-assets.com/books/1328559506m...,divergent divergent 1,0.000041,0.000170
5056,22914373,3,4.000000,My True Love Gave to Me: Twelve Winter Romances,2080,https://www.goodreads.com/book/show/22914373-m...,https://images.gr-assets.com/books/1418488164m...,my true love gave to me twelve winter romances,0.004327,0.017308


**8. Improve The Display Of The Books**

In [47]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

top_recs.style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score
1339,13455782,4,4.500000,"Shatter Me (Shatter Me, #1)",14847,Goodreads,,shatter me shatter me 1,0.001078,0.004849
3174,1809465,3,4.333333,"Rodrick Rules (Diary of a Wimpy Kid, #2)",95364,Goodreads,,rodrick rules diary of a wimpy kid 2,0.000094,0.000409
8748,3293821,3,4.333333,"The Last Straw (Diary of a Wimpy Kid, #3)",85757,Goodreads,,the last straw diary of a wimpy kid 3,0.000105,0.000455
7230,2767052,13,4.307692,"The Hunger Games (The Hunger Games, #1)",4899965,Goodreads,,the hunger games the hunger games 1,0.000034,0.000149
3031,17927395,7,4.285714,"A Court of Mist and Fury (A Court of Thorns and Roses, #2)",120403,Goodreads,,a court of mist and fury a court of thorns and roses 2,0.000407,0.001744
1268,13335037,9,4.111111,"Divergent (Divergent, #1)",1962813,Goodreads,,divergent divergent 1,0.000041,0.000170
5056,22914373,3,4.000000,My True Love Gave to Me: Twelve Winter Romances,2080,Goodreads,,my true love gave to me twelve winter romances,0.004327,0.017308


**_______________K-Nearest Neighbor_____________________**

In [267]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.neighbors import NearestNeighbors

# Step 1: Reading in books we like
my_books = pd.read_csv("liked_books.csv", index_col=0)
my_books["book_id"] = my_books["book_id"].astype(str)

In [268]:
# Step 2: Mapping CSV book IDs to book IDs
csv_book_mapping = {}
with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

# Set that contains all of the unique books that we have read
book_set = set(my_books["book_id"])

In [269]:
# Step 3: Finding similar users using KNN
interactions_list = []
with open("goodreads_interactions.csv") as f:
    while True:
        line = f.readline()
        if not line:
            break

        user_id, csv_id, _, rating, _ = line.strip().split(",")
        book_id = csv_book_mapping.get(csv_id)

        if book_id in book_set:
            interactions_list.append([user_id, book_id, rating])

In [270]:
# Step 4: Creating the user-book matrix
interactions = pd.DataFrame(interactions_list, columns=["user_id", "book_id", "rating"])
interactions["book_id"] = interactions["book_id"].astype(str)
interactions["user_id"] = interactions["user_id"].astype(str)
interactions["rating"] = pd.to_numeric(interactions["rating"])
interactions["user_index"] = interactions["user_id"].astype("category").cat.codes
interactions["book_index"] = interactions["book_id"].astype("category").cat.codes
ratings_mat_coo = coo_matrix((interactions["rating"], (interactions["user_index"], interactions["book_index"])))
ratings_mat = ratings_mat_coo.tocsr()

k = 15  # Number of similar users to find
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(ratings_mat)

my_index = 0
distances, indices = knn_model.kneighbors(ratings_mat[my_index], n_neighbors=k+1)
similar_user_indices = indices.squeeze()[1:]

In [271]:
# Step 5: Filter and process similar users data
similar_users = interactions[interactions["user_index"].isin(similar_user_indices)].copy()
similar_users = similar_users[similar_users["user_id"] != "-1"]

In [276]:
# Step 6: Creating book recommendations
book_recs = similar_users.groupby("book_id").rating.agg(['count', 'mean'])
book_recs.reset_index(inplace=True)

In [275]:
print("Number of rows in book_recs:", len(book_recs))

Number of rows in book_recs: 5


In [260]:
book_recs

,book_id,count,mean
0,17235026,1,0.000000
1,18460392,1,0.000000
2,20910157,2,0.000000
3,9361589,1,0.000000
4,9418327,15,4.133333


In [251]:
# Step 7: Adding book titles
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [252]:
book_recs = book_recs.merge(books_titles, how="inner", on="book_id")

In [253]:
# Step 8: Ranking the book recommendations
book_recs["adjusted_count"] = book_recs["count"] * (book_recs["count"] / book_recs["ratings"])

In [254]:
book_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count
0,17235026,1,0.000000,The Girl with All the Gifts,105967,https://www.goodreads.com/book/show/17235026-t...,https://images.gr-assets.com/books/1403033579m...,the girl with all the gifts,0.000009
1,18460392,1,0.000000,All the Bright Places,142495,https://www.goodreads.com/book/show/18460392-a...,https://images.gr-assets.com/books/1404331702m...,all the bright places,0.000007
2,20910157,2,0.000000,Yes Please,262157,https://www.goodreads.com/book/show/20910157-y...,https://images.gr-assets.com/books/1402815435m...,yes please,0.000015
3,9361589,1,0.000000,The Night Circus,441272,https://www.goodreads.com/book/show/9361589-th...,https://images.gr-assets.com/books/1387124618m...,the night circus,0.000002
4,9418327,15,4.133333,Bossypants,523393,https://www.goodreads.com/book/show/9418327-bo...,https://images.gr-assets.com/books/1481509554m...,bossypants,0.000430


In [255]:
book_recs["score"] = book_recs["mean"] * book_recs["adjusted_count"]

In [256]:
book_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score
0,17235026,1,0.000000,The Girl with All the Gifts,105967,https://www.goodreads.com/book/show/17235026-t...,https://images.gr-assets.com/books/1403033579m...,the girl with all the gifts,0.000009,0.000000
1,18460392,1,0.000000,All the Bright Places,142495,https://www.goodreads.com/book/show/18460392-a...,https://images.gr-assets.com/books/1404331702m...,all the bright places,0.000007,0.000000
2,20910157,2,0.000000,Yes Please,262157,https://www.goodreads.com/book/show/20910157-y...,https://images.gr-assets.com/books/1402815435m...,yes please,0.000015,0.000000
3,9361589,1,0.000000,The Night Circus,441272,https://www.goodreads.com/book/show/9361589-th...,https://images.gr-assets.com/books/1387124618m...,the night circus,0.000002,0.000000
4,9418327,15,4.133333,Bossypants,523393,https://www.goodreads.com/book/show/9418327-bo...,https://images.gr-assets.com/books/1481509554m...,bossypants,0.000430,0.001777


In [257]:
book_recs = book_recs[~book_recs["book_id"].isin(my_books["book_id"])]

In [233]:
book_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score


In [187]:
my_books["mod_title"] = my_books["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True).str.lower()

In [188]:
# Replace any sequences of spaces
my_books["mod_title"] = my_books["mod_title"].str.replace("\s+", " ", regex=True)

In [189]:
# Remove recommendations where the mod title matches the books we already read
book_recs = book_recs[~book_recs["mod_title"].isin(my_books["mod_title"])]

In [190]:
# Remove recommendations that appeared less than twice in our recommendations
book_recs = book_recs[book_recs["count"] > 2]

In [191]:
# Remove recommendations with mean rating below a certain threshold
book_recs = book_recs[book_recs["mean"] >= 4]

In [192]:
# Sort the recommendations based on score
top_recs = book_recs.sort_values("mean", ascending=False)

In [193]:
# Step 9: Improve the display of the books
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

top_recs.style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score
